In [23]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score

### Load dataset and preprocess

In [24]:
df = pd.read_csv('https://raw.githubusercontent.com/ETSISI-OGVD/practicaogvd23-24-team-b/main/data/apple_quality.csv', sep=',')

df.dropna(inplace=True)

df.drop('A_id', axis=1, inplace=True)

df = df[df['Acidity'] != 'Created_by_Nidula_Elgiriyewithana']
df['Acidity'] = df['Acidity'].astype(float)

le = LabelEncoder()
df['Quality'] = le.fit_transform(df['Quality'])

X = df.drop('Quality', axis=1)
y = df['Quality']

Select an element to predict

In [29]:
element_index = 39

infer = X.loc[element_index, :].values.tolist()

print("Data: ", infer)
print("Class: ", y.loc[element_index])

Data:  [1.286737909, -2.429524527, 0.956133102, 1.477542241, 4.786375676, -3.382357404, 2.519346925]
Class:  1


### Inference

In [30]:
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True)

data =  {
  "input_data": {
    "columns": [
      "Size",
      "Weight",
      "Sweetness",
      "Crunchiness",
      "Juiciness",
      "Ripeness",
      "Acidity"
    ],
    "index": [1],
    "data": [infer]
  },
  "params": {}
}

body = str.encode(json.dumps(data))

url = 'https://azure-ml-ws-fltvc.northeurope.inference.ml.azure.com/score'
api_key = 'APaoT4wdIjSP3n43KETgsat15e2j6cyd' # Service deleted (key is not valid)
if not api_key:
    raise Exception("A key should be provided to invoke the endpoint")

headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key), 'azureml-model-deployment': 'sklearn-apple-model-1' }
req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print("Predicted class: ", result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    print(error.info())
    print(error.read().decode("utf8", 'ignore'))



Predicted class:  b'[1]'
